In [1]:
# generate data for testing acc@10
# pick the first 100 ligands from the test_set
# pair all proteins with all ligands
# (100x100 pairs)
# (total of 10,000 pairs...)

In [ ]:
%cd ..

from pathlib import Path
import numpy as np
import pickle as pk
from multiprocessing import Pool
import pandas as pd
from tqdm import tqdm
from htmd.ui import *
from htmd.molecule.voxeldescriptors import *

%load_ext autoreload
%autoreload 2


In [5]:
test_size = 200
base_dest = './data/processed_data/test_acc10'
Path(base_dest).mkdir(exist_ok=True, parents=True)
df = pd.read_csv('./data/csv/test_lig_pro_pairs.csv')
idxs = sorted(list(df.lig_id.unique()))
idxs = idxs[:test_size]

FileNotFoundError: File b'./data/csv/test_lig_pro_pairs.csv' does not exist

In [10]:
dests = []
ligs = []
proteins = []
score = []

for lig_idx in tqdm(idxs):
    lig_src = '../data/processed_data/ligands/' + str(lig_idx).zfill(4) + '.pk'
    with open(lig_src, 'rb') as f:
        lig_features, centers = pk.load(f)
    
    for pro_idx in idxs:
        dest = base_dest + '/' + str(pro_idx).zfill(4) + '_pro_' + str(lig_idx).zfill(4) + '_lig.npy'
        
        if not Path(dest).exists():
        
            mol = Molecule('../data/training_data/'+str(pro_idx).zfill(4)+'_pro_cg.pdb', keepaltloc='all')
            pro_features, _ = getVoxelDescriptors(mol, usercenters=np.array(centers), voxelsize=1, method='NUMBA')
            pro_features = pro_features.reshape(24, 24, 24, pro_features.shape[1])
            combined = np.concatenate((pro_features, lig_features), axis=3)
            combined = combined.astype(np.float32)

            np.save(dest, combined)
            
        dests.append(dest)
        ligs.append(lig_idx)
        proteins.append(pro_idx)
        score.append(1 if lig_idx == pro_idx else 0)
        
        


  0%|          | 0/200 [00:00<?, ?it/s]/home/alex/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \
100%|██████████| 200/200 [9:06:37<00:00, 163.99s/it]


In [11]:
csv_dest = f'../data/csv/test_acc10_{test_size}.csv'
pro_lig_record = pd.DataFrame({'lig_id': ligs, 'pro_id': proteins, 
                               'dest': dests, 'score': score}, index=None)
pro_lig_record.to_csv(csv_dest, index=None)